In [3]:
import pymysql
import pandas as pd
import classified_names

In [4]:
#Dictionary for gender mapping
name_gender = classified_names.output
name_gender['Ararat'] = 'Male'

In [5]:
#Connecting to database
user = 'user_here'
password = 'password_here'
host = 'localhost'
port = 3306
database = 'database_here'
connection = pymysql.connect(user=user, password=password, host=host, port=port, database=database)

In [ ]:
#Reading sql tables
company_t = "select * from company_table"
exec_t = 'select * from executive_table'
act_t = 'select * from activity_table'
prod_t = 'select * from product_table'
company_table = pd.read_sql(company_t, connection)
executive_table = pd.read_sql(exec_t, connection)
activity_table = pd.read_sql(act_t, connection)
product_table = pd.read_sql(prod_t, connection)

In [7]:
#removing empty executives, keeping only companies with one executive
executive_table = executive_table[executive_table['executive'] != 'N/A']
executive_counts = executive_table.groupby('url_id').size()
single_executive_companies = executive_counts[executive_counts == 1]
executive_table = executive_table[executive_table['url_id'].isin(single_executive_companies.index)]

In [8]:
#Splitting executive rows into full name and role columns
executive_table['executive'] = executive_table['executive'].astype('str')
executive_table['role'] = executive_table['executive'].str.split(',', n=1).str[1].str.strip()
executive_table['full_name'] = executive_table['executive'].str.split(',', n=1).str[0].str.strip()

In [9]:
#Function to extract first names for mapping
def extract_first_name(full_name, suffixes=["yan", "ian", "ts", "unc", "jan", "tz", "ians"]):
    if not full_name:
        return None
    
    name_parts = full_name.split()
    current_first_names = []

    for part in name_parts:
        if part.endswith('.'):
            continue  
        elif any(part.endswith(suffix) for suffix in suffixes):
            continue  
        else:
            current_first_names.append(part)

    return ' '.join(current_first_names) if current_first_names else None

executive_table['first_name'] = executive_table['full_name'].apply(extract_first_name)

In [10]:
# Removing rows where first_name is None or empty, mapping gender
executive_table = executive_table[executive_table['first_name'].notna() & (executive_table['first_name'] != '')]
executive_table['gender'] = executive_table['first_name'].map(name_gender)


In [13]:
executive_table = executive_table[executive_table['gender']!= 'Unknown']

In [16]:
#Saving table as pkl file
executive_table.to_pickle('executive_table.pkl')
company_table.to_pickle('company_table.pkl')
activity_table.to_pickle('activity_table.pkl')
product_table.to_pickle('product_table.pkl')